##Libraries&Load Data


In [1]:
pip install -U pymoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210078 sha256=d6523a88a9e4ea8ece21fc941c75053d71076b1e19494207f8be7b1565cd909c
  Stored in directory: /root/.cache/pip/wheels/01/e1/49/37e6bde9886439057450c494a79b0bef8bbe897a54aebfc757
Successfully built grapheme


In [2]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from pymoo.problems import get_problem
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.visualization.scatter import Scatter
from pymoo.decomposition.asf import ASF
from pymoo.mcdm.pseudo_weights import PseudoWeights
from pymoo.problems.functional import FunctionalProblem

import pandas as pd
import numpy as np
import random
import tensorflow as tf
import joblib
import pickle
from scipy.optimize import minimize, LinearConstraint

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/Tesi/data/Dati/'
data_load_path = path + 'data/load/data_load.xlsx'
data_prod_path = path + 'data/production/data_production.xlsx'
data_load = pd.read_excel(data_load_path, decimal=',')
data_prod = pd.read_excel(data_prod_path, decimal=',')
data_load = data_load.set_index("datetime")
data_prod = data_prod.set_index("datetime")

def df_to_X_y(df, window_size):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np) - window_size):
      row = [a for a in df_as_np[i : i + window_size]]
      X.append(row)
      label = df_as_np[i + window_size]
      y.append(label)
  return np.array(X), np.array(y)

win_size = 3
X_load, y_load = df_to_X_y(data_load, win_size)
X_prod, y_prod = df_to_X_y(data_prod, win_size)

y_load = y_load.flatten()
y_prod = y_prod.flatten()

In [ ]:
# Load the models

model_load_path = path + 'models/load/gru_model.keras'
model_load_gru = tf.keras.models.load_model(model_load_path)
model_load_path = path + 'models/production/lstm_model.keras'
model_prod_lstm = tf.keras.models.load_model(model_load_path)

# Make predictions using the loaded model

load = data_load.iloc[3:]
load['pred'] = model_load_gru.predict(X_load).flatten()
load['pred'] = load['pred'].abs()

# Make predictions using the loaded model

prod = data_prod.iloc[3:]
prod['pred'] = model_prod_lstm.predict(X_load).flatten()
prod['pred'] = prod['pred'].abs()

def operation(val):
    return val / 3 * 5
load = load.applymap(operation)
prod = prod.applymap(operation)

##Data Generation

In [354]:
n_prosumers = 3
n_consumers = 2
C_spot = 0.174
C_sell = 0.2
omega_t = [str(ts) for ts in load.index]
#omega_t = 24
omega_alpha = n_prosumers + n_consumers
D = 5
C = 5

In [355]:
# Generation of data

tot_load_m = []
tot_prod_m = []
x = np.array([1, 2, 3, 4, 5])
result = (x / 3) * 5

for t in range(len(omega_t)):
    p = prod.loc[omega_t[t],'pred']
    l = load.loc[omega_t[t], 'pred']
    tot_load = []
    tot_prod = []
    for i in range(n_prosumers):
        if (p > 0):
          deviation = random.uniform(-0.02, 0.02)
          pi = p + deviation

          deviation = random.uniform(-0.02, 0.02)
          ci = l + deviation

          if pi > ci:
              tot_load.append(0)
              tot_prod.append(pi-ci)
          elif pi == ci:
              tot_load.append(0)
              tot_prod.append(0)
          else:
              tot_load.append(ci-pi)
              tot_prod.append(0)
        else:
          tot_prod.append(0)
          deviation = random.uniform(-0.02, 0.02)
          ci = l + deviation
          tot_load.append(ci)

    for i in range(n_consumers):
        deviation = random.uniform(-0.02, 0.02)
        ci = l + deviation
        tot_load.append(ci)
        tot_prod.append(0)

    tot_prod_m.append(tot_prod)
    tot_load_m.append(tot_load)


# Multiobjective optimization

In [356]:
omega_t = 24
omega_alpha = 5
D = 5
C = 5
C_spot = 0.174
C_sell = 0.2
n_var = omega_t*omega_alpha*3
df = pd.DataFrame(columns=['dem', 'res_pv', 'C_spot', 'C_sell', 'D', 'C'])
for t in range(24):
    for h in range(omega_alpha):
        row_values = [tot_load_m[t][h], tot_prod_m[t][h], C_spot, C_sell, D, C]
        df.loc[len(df)] = row_values

In [358]:
#      h0      h1    h2      h3     h4
#     0 1     2 3    4 5    6 7     8 9
# t0  I X     I X    I X    I X     I X
#     10 11  12 13  14 15    18 19  20 21
# t1  I  X   I  X    I X    I X    I X
#     30 31  32 33  34 35   36 37  38 39
# t2  I X    I X     I X    I X    I X
# ...
# t24
#     C_LEM_1 ... C_LEM_24

#   0  1      119   120 121 ... 239
#   I1 I2 ... I120  X1  X2  ... X24  C_LEM1 ... C_LEM24

In [363]:
def objective_function_1(x):
    f1 = 0
    for t in range(omega_t):
      for h in range(omega_alpha):
        f1 += C_spot * (df['dem'].loc[t] - x[t*10+h*2]) + (1.05) * x[-t] * x[t*10+h*2] - C_sell * (df['res_pv'].loc[t] - x[t*10+h*2+1]) - (0.95) * x[-t] * x[t*10+h*2+1]
    return f1

def objective_function_2(x):
    f2 = 0
    for t in range(omega_t):
      for h in range(omega_alpha):
        f2 += -x[t*10+h*2]
    return f2

def constraint_1(x):
  s_X = 0
  s_I = 0
  for t in range(omega_t):
    for h in range(omega_alpha):
      s_I +=x[t*10+h*2]
      s_X += x[t*10+h*2+1]
  return s_X - s_I

def constraint_2(x):
  return x[0] - df['dem'].loc[0]

def constraint_3(x):
  if df['dem'].loc[0] == 0:
    return x[1]
  else:
    return x[1] - df['dem'].loc[0]

objs = [
    objective_function_1,
    objective_function_2
]
c_eq = [constraint_1]
c_ieq = [constraint_2, constraint_3]
n_var = 264
problem = FunctionalProblem(n_var,
                            objs,
                            constr_eq=c_eq,
                            constr_ieq=c_ieq,
                            xl=np.zeros(264),
                            xu=np.full(264, 0.5)
                           )

F, G, H = problem.evaluate(np.random.rand(3, 264))

def multi_optimization(pop_size, n_offsprings, prob, eta_1, eta_2, termination):
    algorithm = NSGA2(
        pop_size=pop_size,
        n_offsprings=n_offsprings,
        sampling=FloatRandomSampling(),
        crossover=SBX(prob=prob, eta=eta_1),
        mutation=PM(eta=eta_2),
        eliminate_duplicates=True
    )

    termination = get_termination("n_gen", termination)

    res = minimize(problem,
                  algorithm,
                  termination,
                  seed=1,
                  return_least_infeasible=True,
                  save_history=False,
                  verbose=False)

    X = res.X
    F = res.F
    fl = F.min(axis=0)
    fu = F.max(axis=0)
    approx_ideal = F.min(axis=0)
    approx_nadir = F.max(axis=0)
    nF = (F - approx_ideal) / (approx_nadir - approx_ideal)

    fl_2 = nF.min(axis=0)
    fu_2 = nF.max(axis=0)
    weights = np.array([0.5, 0.5])
    decomp = ASF()
    i = decomp.do(nF, 1/weights).argmin()
    i_2 = PseudoWeights(weights).do(nF)
    return X[i]

In [364]:
results_multiobj = pd.DataFrame(columns=['G', 'E', 'I_p2p', 'X_p2p', 'G+I', 'dem', 'E+X', 'res_pv', 'C_LEM'])
pop_size = 100
n_offsprings = 50
prob = 0.9
eta_1 = 15
eta_2 = 15
termination = 50
omega_t = 24
omega_alpha = 5
result = multi_optimization(pop_size, n_offsprings, prob, eta_1, eta_2, termination)
variables = pd.DataFrame(columns=['I', 'X','C_LEM'])
for t in range(omega_t):
    for h in range(omega_alpha):
        I = result[t*10+h*2]
        X = result[t*10+h*2+1]
        C_LEM = result[-t]
        row_values = [I, X, C_LEM]
        variables.loc[len(variables)] = row_values
merged_df = pd.merge(df, variables, left_index=True, right_index=True, how='inner')
merged_df['G'] = merged_df['dem'] - merged_df['I']
merged_df['E'] = merged_df['res_pv'] - merged_df['X']
merged_df

<ipython-input-363-0a01ee15a5d3>:76: RuntimeWarning: invalid value encountered in divide
  nF = (F - approx_ideal) / (approx_nadir - approx_ideal)


,dem,res_pv,C_spot,C_sell,D,C,I,X,C_LEM,G,E
0,0.038165,0.0,0.174,0.2,5.0,5.0,0.022539,0.014822,0.022539,0.015626,-0.014822
1,0.048474,0.0,0.174,0.2,5.0,5.0,0.205746,0.307869,0.022539,-0.157272,-0.307869
2,0.037405,0.0,0.174,0.2,5.0,5.0,0.176990,0.447552,0.022539,-0.139585,-0.447552
3,0.212941,0.0,0.174,0.2,5.0,5.0,0.324257,0.464084,0.022539,-0.111315,-0.464084
4,0.211688,0.0,0.174,0.2,5.0,5.0,0.352479,0.435021,0.022539,-0.140792,-0.435021
...,...,...,...,...,...,...,...,...,...,...,...
115,0.024940,0.0,0.174,0.2,5.0,5.0,0.092863,0.390131,0.474063,-0.067923,-0.390131
116,0.043236,0.0,0.174,0.2,5.0,5.0,0.251166,0.184927,0.474063,-0.207930,-0.184927
117,0.017557,0.0,0.174,0.2,5.0,5.0,0.162114,0.007937,0.474063,-0.144557,-0.007937
118,0.213105,0.0,0.174,0.2,5.0,5.0,0.441597,0.248352,0.474063,-0.228492,-0.248352


In [365]:
i = 20
print((merged_df['dem'].loc[i] - merged_df['I'].loc[i]) + merged_df['I'].loc[i] + merged_df['D'].loc[i] + merged_df['res_pv'].loc[i])
print((merged_df['res_pv'].loc[i] - merged_df['X'].loc[i]) + merged_df['X'].loc[i] + merged_df['C'].loc[i] + merged_df['dem'].loc[i])

5.062399427563977
5.062399427563977


In [366]:
sum(merged_df['I'])

30.785091477371697

In [367]:
sum(merged_df['X'])

30.783627853617293